# Task2 
## tashk 2.1 y 2.2: 
Definir las variables y restringir días así como la relación para los estudiantes y los cursos que tienen

In [26]:
import random
import time
import math

# Definir las variables y dominios
examenes = ["Quimica", "Biología", "Matemáticas", "Física", "Algoritmos", "Diseño de datos", "Literatura"]
dias = ["Lunes", "Martes", "Miercoles"]

# Relaciones entre estudiantes y exámenes
estudiantes_examenes = [
    ("Estudiante1", "Quimica"),
    ("Student", "Biología"),
    ("Student", "Diseño de datos"),
    ("Estudiante2", "Biología"),
    ("Estudiante2", "Matemáticas"),
    ("Estudiante2", "Diseño de datos"),
    ("Estudiante3", "Matemáticas"),
    ("Estudiante3", "Física"),
    ("Estudiante3", "Literatura"),
    ("Estudiante4", "Física"),
    ("Estudiante4", "Algoritmos"),
    ("Estudiante4", "Literatura")
]

### 2.3 backtraking search

In [68]:
def backtrack(assigned_tests, student_index):
  """
  This function recursively assigns tests to students using backtracking.

  Args:
      assigned_tests: A dictionary where keys are students and values are lists of assigned tests.
      student_index: The index of the current student being processed.

  Returns:
      True if a valid assignment is found, False otherwise.
  """
  if student_index == len(estudiantes_examenes):
    # All students have been assigned tests, return True
    return True

  student, exam = estudiantes_examenes[student_index]
   # Check if exam can be assigned to any of the three days
  for day in dias:
    if is_valid_assignment(assigned_tests, student, exam, day):
      # Assign the exam to the student for that day
      assigned_tests.setdefault(student, []).append((day, exam))
      
      # Try assigning tests to the next student
      if backtrack(assigned_tests, student_index + 1):
        return True

      # Backtrack if assignment didn't lead to a solution, remove assigned exam
      assigned_tests[student].pop()

  return False


def is_valid_assignment(assigned_tests, student, exam, day):
  """
  This function checks if assigning a test to a student on a specific day is valid.

  Args:
      assigned_tests: A dictionary where keys are students and values are lists of assigned tests.
      student: The student for whom the assignment is being considered.
      exam: The exam being assigned.
      day: The day on which the exam is being considered.

  Returns:
      True if the assignment is valid, False otherwise.
  """
  # Check if student already has a test assigned on that day
  if student in assigned_tests and any(assigned_day == day for assigned_day, _ in assigned_tests[student]):
    return False

  # Check if another student has the same exam assigned on that day
  for other_student, other_exams in assigned_tests.items():
    if student != other_student and (exam,day) in other_exams:
      return False

  return True


In [69]:

assigned_tests = {}
if backtrack(assigned_tests, 0):
  # Print the assigned tests
  for student, student_tests in assigned_tests.items():
    print(f"Student: {student}")
    for day, exam in student_tests:
      print(f"\t- {day}: {exam}")
else:
  print("No valid assignment found for all students.")


Student: Estudiante1
	- Lunes: Quimica
	- Martes: Biología
	- Miercoles: Diseño de datos
Student: Estudiante2
	- Lunes: Biología
	- Martes: Matemáticas
	- Miercoles: Diseño de datos
Student: Estudiante3
	- Lunes: Matemáticas
	- Martes: Física
	- Miercoles: Literatura
Student: Estudiante4
	- Lunes: Física
	- Martes: Algoritmos
	- Miercoles: Literatura


### 2.4 beam search algorithm

In [39]:
def beam_search_exam_calendars(examenes, estudiantes_examenes, dias, beam_width):
    calendars = {}  # Diccionario para almacenar los calendarios de exámenes de cada estudiante
    
    # Iterar sobre cada estudiante
    for estudiante in set([estudiante for estudiante, examen in estudiantes_examenes]):
        # Filtrar los exámenes específicos para este estudiante
        exames_estudiante = [examen for est, examen in estudiantes_examenes if est == estudiante]
        # Inicializar el conjunto de soluciones con una solución vacía
        soluciones = [{}]
        # Iterar a través de cada examen del estudiante
        for examen in exames_estudiante:
            nuevas_soluciones = []
            # Expandir cada solución candidata en el conjunto actual
            for asignacion in soluciones:
                # Probar cada día posible para el examen actual
                for dia in dias:
                    # Verificar si el día satisface las restricciones
                    if dia_valido(estudiante, examen, dia, asignacion, estudiantes_examenes):
                        # Crear una nueva asignación basada en la asignación actual y el nuevo examen
                        nueva_asignacion = asignacion.copy()
                        nueva_asignacion[examen] = dia
                        # Agregar la nueva asignación al conjunto de nuevas soluciones
                        nuevas_soluciones.append(nueva_asignacion)
            
            # Seleccionar las mejores soluciones basadas en una heurística (cantidad de restricciones incumplidas)
            soluciones = sorted(nuevas_soluciones, key=lambda x: contar_restricciones_incumplidas(x, estudiantes_examenes))[:beam_width]
        
        # Seleccionar la mejor solución encontrada para este estudiante
        mejor_solucion = min(soluciones, key=lambda x: contar_restricciones_incumplidas(x, estudiantes_examenes))
        calendars[estudiante] = mejor_solucion
    
    return calendars

# Función para verificar si un día satisface las restricciones para un estudiante
def dia_valido(estudiante, examen, dia, asignacion, estudiantes_examenes):
    # Verificar restricción: Todos los exámenes en días diferentes
    if dia in asignacion.values():
        return False
    
    # Verificar restricción: Ningún estudiante tiene más de un examen por día
    estudiantes_con_examen = [estudiante for estudiante, examen_asignado in asignacion.items() if examen_asignado == dia]
    if len(estudiantes_con_examen) > 1:
        return False
    
    # Verificar restricción: Los estudiantes que toman el mismo curso no pueden tener exámenes programados para el mismo día
    for otro_estudiante, otro_examen in asignacion.items():
        if estudiante != otro_estudiante and (estudiante, examen) in estudiantes_examenes and (otro_estudiante, otro_examen) in estudiantes_examenes:
            return False
    
    return True

# Función para contar la cantidad de restricciones incumplidas en una asignación
def contar_restricciones_incumplidas(asignacion, estudiantes_examenes):
    restricciones_incumplidas = 0
    
    # Verificar restricción: Todos los exámenes en días diferentes
    dias_asignados = list(asignacion.values())
    dias_unicos = set(dias_asignados)
    restricciones_incumplidas += len(dias_asignados) - len(dias_unicos)
    
    # Verificar restricción: Ningún estudiante tiene más de un examen por día
    for dia in dias_unicos:
        estudiantes_con_examen = [estudiante for estudiante, examen_asignado in asignacion.items() if examen_asignado == dia]
        restricciones_incumplidas += max(0, len(estudiantes_con_examen) - 1)
    
    # Verificar restricción: Los estudiantes que toman el mismo curso no pueden tener exámenes programados para el mismo día
    for estudiante, examen_asignado in asignacion.items():
        for otro_estudiante, otro_examen in asignacion.items():
            if estudiante != otro_estudiante and (estudiante, otro_examen) in estudiantes_examenes and (otro_estudiante, examen_asignado) in estudiantes_examenes:
                restricciones_incumplidas += 1
    
    return restricciones_incumplidas

In [41]:
beam_width = 3  
calendarios_examenes = beam_search_exam_calendars(examenes, estudiantes_examenes, dias, beam_width)
for estudiante, calendario in calendarios_examenes.items():
    print(f"Calendario de exámenes para {estudiante}: {calendario}")

Calendario de exámenes para Estudiante4: {'Física': 'Lunes', 'Algoritmos': 'Martes', 'Literatura': 'Miercoles'}
Calendario de exámenes para Estudiante3: {'Matemáticas': 'Lunes', 'Física': 'Martes', 'Literatura': 'Miercoles'}
Calendario de exámenes para Estudiante1: {'Quimica': 'Lunes', 'Biología': 'Martes', 'Diseño de datos': 'Miercoles'}
Calendario de exámenes para Estudiante2: {'Biología': 'Lunes', 'Matemáticas': 'Martes', 'Diseño de datos': 'Miercoles'}


### 2.5 local search algorithm

In [3]:
# Asignar un día inicial aleatorio a cada examen
solucion_inicial = {examen: random.choice(dias) for examen in examenes}

# Definir la función de restricciones
def restricciones(solucion):
    # Comprobar que todos los exámenes se realicen en días diferentes
    if len(set(solucion.values())) < len(examenes):
        return False
    
    # Comprobar que ningún estudiante tenga más de un examen por día
    for estudiante in set(estudiante for estudiante, examen in estudiantes_examenes):
        dias_examenes_estudiante = [solucion[examen] for estudiante_, examen in estudiantes_examenes if estudiante_ == estudiante]
        if len(dias_examenes_estudiante) != len(set(dias_examenes_estudiante)):
            return False
    
    # Comprobar que los estudiantes que toman el mismo curso no tengan exámenes el mismo día
    for estudiante1, Quimica in estudiantes_examenes:
        for estudiante2, Biología in estudiantes_examenes:
            if Quimica == Biología and estudiante1 != estudiante2:
                if solucion[Quimica] == solucion[Biología]:
                    return False

    return True

# Función de evaluación para medir qué tan buena es una solución
def evaluar_solucion(solucion):
    violaciones = 0
    for estudiante in set(estudiante for estudiante, examen in estudiantes_examenes):
        dias_examenes_estudiante = [solucion[examen] for estudiante_, examen in estudiantes_examenes if estudiante_ == estudiante]
        violaciones += len(dias_examenes_estudiante) - len(set(dias_examenes_estudiante))
    return violaciones

# Algoritmo de búsqueda local mejorado con enfriamiento simulado
def local_search_enhanced(solucion):
    iteraciones = 0
    inicio = time.time()
    temperatura = 1.0
    tasa_enfriamiento = 0.95
    mejor_solucion = solucion.copy()
    mejor_evaluacion = evaluar_solucion(mejor_solucion)
    while temperatura > 0.01 and iteraciones < 10000:
        examen_aleatorio = random.choice(examenes)
        dia_aleatorio = random.choice(dias)
        solucion_anterior = solucion.copy()
        solucion[examen_aleatorio] = dia_aleatorio
        evaluacion_actual = evaluar_solucion(solucion)
        if evaluacion_actual < mejor_evaluacion:
            mejor_solucion = solucion.copy()
            mejor_evaluacion = evaluacion_actual
        elif math.exp((mejor_evaluacion - evaluacion_actual) / temperatura) > random.random():
            solucion = solucion_anterior
        temperatura *= tasa_enfriamiento
        iteraciones += 1
    fin = time.time()
    tiempo_ejecucion = fin - inicio
    return mejor_solucion, tiempo_ejecucion

# Solución
solucion_final, tiempo_ejecucion = local_search_enhanced(solucion_inicial)
print("Solución final:", solucion_final)
print("Tiempo de ejecución:", tiempo_ejecucion, "segundos")


Solución final: {'Quimica': 'Miercoles', 'Biología': 'Martes', 'Matemáticas': 'Miercoles', 'Física': 'Martes', 'Algoritmos': 'Lunes', 'Diseño de datos': 'Lunes', 'Literatura': 'Miercoles'}
Tiempo de ejecución: 0.0009756088256835938 segundos


El algoritmo de búsqueda local demostró ser rápido, con un tiempo de ejecución de aproximadamente 0.000 segundos para el problema de programación de exámenes,fue bastante óptimo encontrando la solución.